In [1]:
%matplotlib inline
import logging;logging.basicConfig(level=logging.INFO)
import sys
import pandas as pd

sys.path.append("../")
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric

from IPython.display import Markdown, display
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
dataset_used = "german" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
    dataset_orig = AdultDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
    
elif dataset_used == "german":
    dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
    
elif dataset_used == "compas":
    dataset_orig = CompasDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]    

In [3]:
import logictensornetworks2 as ltn
import logictensornetworks2.fuzzy_ops as fuzzy_ops
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

In [4]:
Not = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Not_Std())
And = ltn.Wrapper_ConnectiveOp(fuzzy_ops.And_Prod())
Or = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Implies_Reichenbach())
Equiv = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Equiv(And,Implies))
Forall = ltn.experimental.Wrapper_AggregationOp(ltn.experimental.Aggreg_pMeanError(p=5))
Exists = ltn.experimental.Wrapper_AggregationOp(ltn.experimental.Aggreg_pMean(p=5))
scale_orig = StandardScaler()

In [5]:
def f(input):
      var_data = ltn.variable("input", input)
      result = D(var_data)
      return result.numpy()

class Predpred(object):
  def __init__(self, oracle):
        self.oracle = oracle
  def predict(self, data):
      var_data = ltn.variable("input", data)
      result = self.oracle(var_data)
      y_test_pred_prob = result.numpy()
      class_thresh = 0.5
      y_test_pred = np.zeros_like(y_test_pred_prob)
      y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig.favorable_label
      y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig.unfavorable_label
      return y_test_pred

In [6]:
kf = KFold(n_splits=5)
Xs = scale_orig.fit_transform(dataset_orig.features)
ys = dataset_orig.labels.ravel()

In [7]:
D = ltn.Predicate.MLP([58],hidden_layer_sizes=(50,25))
trainable_variables = D.trainable_variables
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=5)

kv = {}
for k, (train, test) in enumerate(kf.split(Xs, ys)):
    kv["posits{0}".format(k)] = Xs[train][ys[train]==1].astype(np.float32)
    kv["negats{0}".format(k)] = Xs[train][ys[train]==2].astype(np.float32)
    kv["Xtrain{0}".format(k)] = Xs[train].astype(np.float32)
    kv["ytrain{0}".format(k)] = ys[train].astype(np.float32)
    kv["Xtest{0}".format(k)] = Xs[test].astype(np.float32)
    kv["ytest{0}".format(k)] = ys[test].astype(np.float32)
    
    var_posit = ltn.variable("posits",kv["posits{0}".format(k)])
    var_negat = ltn.variable("negats",kv["negats{0}".format(k)])
    oracle = Predpred(D)
    
    @tf.function
    @ltn.domains()
    def axioms():
        axioms = []
        weights = []
        # forall data_A: A(data_A)
        axioms.append(Forall(ltn.bound(var_posit), D(var_posit)))
        # forall data_B: B(data_B)
        axioms.append(Forall(ltn.bound(var_negat), Not(D(var_negat))))
        axioms = tf.stack([tf.squeeze(ax) for ax in axioms])
        sat_level = formula_aggregator(axioms)
        return sat_level, axioms

    for epoch in range(1000):
        with tf.GradientTape() as tape:
            loss = 1. - axioms()[0]
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        if epoch%200 == 0:
            print("Epoch %d: Sat Level %.3f"%(epoch, axioms()[0]),
                  "Epoch %d: Train-Accuracy %.3f"%(epoch, 
                                                   accuracy_score(kv["ytrain{0}".format(k)],
                                                                  oracle.predict(kv["Xtrain{0}".format(k)]))))
    print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, axioms()[0]))
    kv["test_acc{0}".format(k)] = accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))
    
    dataset_orig_test_pred = dataset_orig.subset(test).copy(deepcopy=True)
    dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.subset(test).features))
    classified_metric_debiasing_test = ClassificationMetric(dataset_orig.subset(test), 
                                                     dataset_orig_test_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)
    
    kv["Disparate_impact{0}".format(k)] = classified_metric_debiasing_test.disparate_impact()
    kv["Parity_Difference{0}".format(k)] = classified_metric_debiasing_test.statistical_parity_difference()

Epoch 0: Sat Level 0.433 Epoch 0: Train-Accuracy 0.542
Epoch 200: Sat Level 0.996 Epoch 200: Train-Accuracy 1.000
Epoch 400: Sat Level 0.999 Epoch 400: Train-Accuracy 1.000
Epoch 600: Sat Level 0.999 Epoch 600: Train-Accuracy 1.000
Epoch 800: Sat Level 1.000 Epoch 800: Train-Accuracy 1.000
Training finished at Epoch 999 with Sat Level 1.000
Epoch 0: Sat Level 0.415 Epoch 0: Train-Accuracy 0.926
Epoch 200: Sat Level 0.518 Epoch 200: Train-Accuracy 0.983
Epoch 400: Sat Level 0.562 Epoch 400: Train-Accuracy 0.990
Epoch 600: Sat Level 0.562 Epoch 600: Train-Accuracy 0.990
Epoch 800: Sat Level 0.562 Epoch 800: Train-Accuracy 0.990
Training finished at Epoch 999 with Sat Level 0.562
Epoch 0: Sat Level 0.516 Epoch 0: Train-Accuracy 0.976
Epoch 200: Sat Level 0.541 Epoch 200: Train-Accuracy 0.988
Epoch 400: Sat Level 0.541 Epoch 400: Train-Accuracy 0.988
Epoch 600: Sat Level 0.541 Epoch 600: Train-Accuracy 0.988
Epoch 800: Sat Level 0.541 Epoch 800: Train-Accuracy 0.988
Training finished at Ep

Average 5-fold accuracy as in Padala and Gujar

In [8]:
(kv["test_acc0"]+kv["test_acc1"]+kv["test_acc2"]+kv["test_acc3"]+kv["test_acc4"])/5

0.885

Average 5-fold Disparity Impact

In [9]:
(kv["Disparate_impact0"]+kv["Disparate_impact1"]+kv["Disparate_impact2"]+kv["Disparate_impact3"]+kv["Disparate_impact4"])/5

0.9039553892656415

Average 5-fold Parity Difference

In [10]:
(kv["Parity_Difference0"]+kv["Parity_Difference1"]+kv["Parity_Difference2"]+kv["Parity_Difference3"]+kv["Parity_Difference4"])/5

-0.07063190582820023

In [11]:
dataset_orig_test_pred = dataset_orig.copy(deepcopy=True)
dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.features))
display(Markdown("#### Model - with bias - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig, 
                                                 dataset_orig_test_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Dataset: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
print("Dataset: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Dataset: Parity Difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())

#### Model - with bias - classification metrics

Dataset: Classification accuracy = 0.970000
Dataset: Disparate impact = 0.897361
Dataset: Parity Difference = -0.075269


In [12]:
X_df = pd.DataFrame(Xs,columns=dataset_orig.feature_names)
X_r_preds = f(np.asarray(Xs).astype(np.float32))
X_df['customer_risk_pred'] = X_r_preds
X_female_df = X_df[X_df['sex'] == X_df['sex'].unique()[0]]
X_male_df = X_df[X_df['sex'] == X_df['sex'].unique()[1]]
X_female_df['customer class'] = pd.qcut(X_female_df['customer_risk_pred'],4, labels=[0,1,2],duplicates='drop')
X_male_df['customer class'] = pd.qcut(X_male_df['customer_risk_pred'],4, labels=[0,1,2],duplicates='drop')
X_inp = pd.concat([X_male_df,X_female_df])
class1F = X_inp[X_inp['sex'] == X_inp['sex'].unique()[0]][X_inp['customer class'] == 0]
class2F = X_inp[X_inp['sex'] == X_inp['sex'].unique()[0]][X_inp['customer class'] == 1]
class3F = X_inp[X_inp['sex'] == X_inp['sex'].unique()[0]][X_inp['customer class'] == 2]

class1M = X_inp[X_inp['sex'] == X_inp['sex'].unique()[1]][X_inp['customer class'] == 0]
class2M = X_inp[X_inp['sex'] == X_inp['sex'].unique()[1]][X_inp['customer class'] == 1]
class3M = X_inp[X_inp['sex'] == X_inp['sex'].unique()[0]][X_inp['customer class'] == 2]

C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:10: Us

In [13]:
inpclass1f = class1F.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass2f = class2F.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass3f = class3F.iloc[:,:-2].astype(np.float32).to_numpy()

inpclass1m = class1M.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass2m = class2M.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass3m = class3M.iloc[:,:-2].astype(np.float32).to_numpy()

var_class1f = ltn.variable("?class1F",inpclass1f)
var_class2f = ltn.variable("?class2F",inpclass2f)
var_class3f = ltn.variable("?class3F",inpclass3f)

var_class1m = ltn.variable("?class1M",inpclass1m)
var_class2m = ltn.variable("?class2M",inpclass2m)
var_class3m = ltn.variable("?class3M",inpclass3m)

In [21]:
D = ltn.Predicate.MLP([58],hidden_layer_sizes=(50,25))
trainable_variables = D.trainable_variables
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=5)

In [22]:
kv = {}
for k, (train, test) in enumerate(kf.split(Xs, ys)):
    kv["posits{0}".format(k)] = Xs[train][ys[train]==1].astype(np.float32)
    kv["negats{0}".format(k)] = Xs[train][ys[train]==2].astype(np.float32)
    kv["Xtrain{0}".format(k)] = Xs[train].astype(np.float32)
    kv["ytrain{0}".format(k)] = ys[train].astype(np.float32)
    kv["Xtest{0}".format(k)] = Xs[test].astype(np.float32)
    kv["ytest{0}".format(k)] = ys[test].astype(np.float32)
    

    var_posit = ltn.variable("posits",kv["posits{0}".format(k)])
    var_negat = ltn.variable("negats",kv["negats{0}".format(k)])
    formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=2)

    @tf.function
    @ltn.domains()
    def axioms():
        axioms = []
        weights = []
        # forall data_A: A(data_A)
        axioms.append(Forall(ltn.bound(var_posit), D(var_posit)))
        weights.append(2.)
        # forall data_B: B(data_B)
        axioms.append(Forall(ltn.bound(var_negat), Not(D(var_negat))))
        weights.append(2.)
        # Equality Constraints
        axioms.append(Forall(ltn.bound(var_class1f,var_class1m), Equiv(D(var_class1f),D(var_class1m)),p=2))
        weights.append(1.5)
        axioms.append(Forall(ltn.bound(var_class2f,var_class2m), Equiv(D(var_class2f),D(var_class2m)),p=2))
        weights.append(1.5)
        axioms.append(Forall(ltn.bound(var_class3f,var_class3m), Equiv(D(var_class3f),D(var_class3m)),p=2))
        weights.append(1.5)
        axioms = tf.stack([tf.squeeze(ax) for ax in axioms])
        weights = tf.stack(weights)
        weighted_axioms = weights*axioms
        sat_level = formula_aggregator(weighted_axioms)
        return sat_level, axioms

    for epoch in range(1000):
        with tf.GradientTape() as tape:
            loss = 1. - axioms()[0]
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        if epoch%200 == 0:
            print("Epoch %d: Sat Level %.3f"%(epoch, axioms()[0]),
                  "Epoch %d: Train-Accuracy %.3f"%(epoch, 
                                                   accuracy_score(kv["ytrain{0}".format(k)],
                                                                  oracle.predict(kv["Xtrain{0}".format(k)]))))
    print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, axioms()[0]),"Epoch %d: Test-Accuracy %.3f"%(epoch, accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))))
    kv["test_acc{0}".format(k)] = accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))
    
    dataset_orig_test_pred = dataset_orig.subset(test).copy(deepcopy=True)
    dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.subset(test).features))
    classified_metric_debiasing_test = ClassificationMetric(dataset_orig.subset(test), 
                                                     dataset_orig_test_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)
    
    kv["Disparate_impact{0}".format(k)] = classified_metric_debiasing_test.disparate_impact()
    kv["Parity_Difference{0}".format(k)] = classified_metric_debiasing_test.statistical_parity_difference()

Epoch 0: Sat Level 0.846 Epoch 0: Train-Accuracy 0.970
Epoch 200: Sat Level 0.982 Epoch 200: Train-Accuracy 0.970
Epoch 400: Sat Level 0.985 Epoch 400: Train-Accuracy 0.970
Epoch 600: Sat Level 0.986 Epoch 600: Train-Accuracy 0.970
Epoch 800: Sat Level 0.986 Epoch 800: Train-Accuracy 0.970
Training finished at Epoch 999 with Sat Level 0.986 Epoch 999: Test-Accuracy 0.970
Epoch 0: Sat Level 0.856 Epoch 0: Train-Accuracy 0.964
Epoch 200: Sat Level 0.985 Epoch 200: Train-Accuracy 0.964
Epoch 400: Sat Level 0.986 Epoch 400: Train-Accuracy 0.964
Epoch 600: Sat Level 0.987 Epoch 600: Train-Accuracy 0.964
Epoch 800: Sat Level 0.987 Epoch 800: Train-Accuracy 0.964
Training finished at Epoch 999 with Sat Level 0.987 Epoch 999: Test-Accuracy 0.995
Epoch 0: Sat Level 0.871 Epoch 0: Train-Accuracy 0.963
Epoch 200: Sat Level 0.979 Epoch 200: Train-Accuracy 0.963
Epoch 400: Sat Level 0.979 Epoch 400: Train-Accuracy 0.963
Epoch 600: Sat Level 0.980 Epoch 600: Train-Accuracy 0.963
Epoch 800: Sat Level

#### Average Test Accuracy

In [23]:
(kv["test_acc0"]+kv["test_acc1"]+kv["test_acc2"]+kv["test_acc3"]+kv["test_acc4"])/5

0.97

#### Average Demographic Parity

In [24]:
(kv["Disparate_impact0"]+kv["Disparate_impact1"]+kv["Disparate_impact2"]+kv["Disparate_impact3"]+kv["Disparate_impact4"])/5

0.9024580904385917

#### Average Disparate Impact

In [25]:
(kv["Parity_Difference0"]+kv["Parity_Difference1"]+kv["Parity_Difference2"]+kv["Parity_Difference3"]+kv["Parity_Difference4"])/5

-0.07230820743333258

In [26]:
dataset_orig_test_pred = dataset_orig.copy(deepcopy=True)
dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.features))
display(Markdown("#### Model - without bias - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig, 
                                                 dataset_orig_test_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Dataset: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
print("Dataset: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Dataset: Parity Difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())

#### Model - without bias - classification metrics

Dataset: Classification accuracy = 0.970000
Dataset: Disparate impact = 0.897361
Dataset: Parity Difference = -0.075269
